# L'Analyse en Composantes Principales ☕️☕️☕️

**<span style='color:blue'> Objectifs de la séquence</span>** 
* Être sensibilisé&nbsp;:
    * aux enjeux mathématiques de l'ACP.
* Être capable&nbsp;:
    * d'implémenter une ACP avec $\texttt{sklearn}$.



 ----

## I. Introduction

L'Analyse en Composante Principale (ACP) consiste à extraire des directions dans lesquelles les données s'étalent particulièrement (i.e. la variance y est maximale). Prenons un exemple. Nous disposons d'un jeu de données dont on aimerait extraire l'information la plus "representative" (dans un sens que nous allons expliciter plus loin). Nous choisissons ici un jeu de données tiré selon une loi gaussienne multivariée $\boldsymbol{x} \sim \mathcal{N}(\boldsymbol{0}, \boldsymbol{\Sigma})$ avec $\boldsymbol{\Sigma} \in \mathcal{S}^+(\mathbb{R}^d)$ (i.e. matrice semi-définie positive de dimension $d\times d$) la matrice de variance/co-variance des données. Affichons le jeu de données.

In [ ]:
import numpy as np
%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12.0, 8.0)
plt.style.use('ggplot')

In [ ]:
def plot(X, X_rec=None, vec=None, color="blue", circles = False):
    assert X.shape[1] == 2, "Dimension must be equal to 2 to plot stuffs."
    eps = 0.1
    plt.scatter(X[:,0],X[:,1], edgecolors=color, facecolors=color)
    x1min_ = X[1].min() - eps
    x1max_ = X[1].max() - eps
    x0min_ = X[0].min() - eps
    x0max_ = X[0].max() + eps
    if vec is not None:
        mean = X.mean(axis=0)
        plt.arrow(
            mean[0], mean[1], vec[0][0], vec[0][1], 
            head_width=0.1, head_length=0.1, fc='g', 
            ec='g', width=0.025, zorder=2
        )
        plt.arrow(
            mean[0], mean[1], vec[1][0], vec[1][1], 
            head_width=0.1, head_length=0.1, fc='r', 
            ec='r', width=0.025, zorder=2
        )
    if X_rec is not None:
        plt.scatter(X_rec[:,0],X_rec[:,1], edgecolors='r', facecolors='none')
        for i in range(X.shape[0]):
            plt.plot(
                [X[i,0], X_rec[i,0]], [X[i,1], X_rec[i,1]], color='r', 
                linestyle = 'dashed', linewidth='0.5'
            )

    plt.xlim((x0min_, x0max_))
    plt.ylim((x1min_, x1max_))
    plt.axis('equal')

In [ ]:
def random_rotation(d):
    Q, _ =  np.linalg.qr(np.random.random((d,d)))
    return Q

def sample_multivariate_gaussian_data(N, d, variance_decay=0.1):
    sigma = np.diag([np.power(variance_decay, i) for i in range(d)])
    Q = random_rotation(d)
    sigma_r = np.dot(np.dot(Q, sigma ), Q.T)
    return np.random.multivariate_normal(np.ones(d), sigma_r, N)

X = sample_multivariate_gaussian_data(500, 2)
print(X.shape)

plot(X, vec = np.eye(X.shape[1]))
plt.show()

**<span style='color:blue'> Question </span>** 
**Dans le code ci-dessus, à quoi sert à la décomposition QR ?** 



 ----


Dans ce cas précis, chaque point est représenté par deux nombres. Que pourrions-nous faire pour ne représenter chaque point par seulement un nombre en perdant le moins d'information possible sur le signal d'origine ? Nous pourrions ainsi chercher à trouver la direction dans l'espace des données telle que les données soient le plus "étalées". Plus formellement on cherche un vecteur unitaire $\boldsymbol{v} \in \mathbb{R}^2$ tel que les projections des données sur $z_i = \langle \boldsymbol{v}, \boldsymbol{x}_i \rangle$ constituent un échantillon transformé dont la variance empirique soit maximale. Il s'agit de résoudre le problème d'optimisation suivant&nbsp;:

$$\underset{\boldsymbol{v}}{argmax}\Bigg[\frac{1}{N} \sum_{i=1}^N  (z_i - \bar{z})^2 \Bigg] = \underset{\boldsymbol{v}}{argmax} \Big[ \boldsymbol{v}^t\bar{\boldsymbol{X}}^T\bar{\boldsymbol{X}}\boldsymbol{v} \Big]$$

sous contrainte que $\lVert\boldsymbol{v}\rVert_2 = 1$ et avec $\bar{z} = \frac{1}{N}\sum_i z_i$ et $\bar{\boldsymbol{X}} \in \mathbb{R}^{n\times d}$, la matrice de *design* (i.e. qui contient nos données) centrée.

Sans la contrainte $\lVert\boldsymbol{v}\rVert_2 = 1$, le problème d'optimisation serait mal posé car la solution ne serait pas unique (il suffirait simplement d'augmenter arbitrairement la norme de n'importe quel vecteur pour avoir une valeur de la fonction objectif arbitrairement grande).


Nous allons montrer que ce problème d'optimisation est strictement équivalent a celui de trouver le vecteur propre de la matrice de covariance $\boldsymbol{\Sigma} = \bar{\boldsymbol{X}}^T\bar{\boldsymbol{X}}$ correspondant à la valeur propre maximale $\lambda_{\max}$.

**<span style='color:blue'> Question </span>** 
**Montrer l'égalité suivante&nbsp;:**

$$\frac{1}{N} \sum_{i=1}^N  (z_i - \bar{z})^2=\boldsymbol{v}^t\bar{\boldsymbol{X}}^T\bar{\boldsymbol{X}}\boldsymbol{v}$$



 ----


## II. Quelques rappels d'algèbre linéaire


### A. Décomposition en valeurs et vecteurs propres

**Qu'est ce qu'un vecteur propre ?** 
Algébriquement, un vecteur propre d'une matrice $\boldsymbol{A}$ est un vecteur tel que&nbsp;:
\begin{equation*}
    \boldsymbol{A}\boldsymbol{v} = \lambda \boldsymbol{v}
\end{equation*}

L'interprétation géométrique est donc qu'il s'agit d'un vecteur dont la direction de son image par l'application linéaire associée à la matrice $\boldsymbol{A}$ n'est qu'une [homothétie](https://fr.wikipedia.org/wiki/Homothétie) (i.e. la direction est inchangée et le vecteur n'est qu'étiré). Il est simplement étiré d'un facteur $\lambda$ qu'on appel sa valeur propre associée.

![Vecteur propre](https://upload.wikimedia.org/wikipedia/commons/thumb/5/58/Eigenvalue_equation.svg/2880px-Eigenvalue_equation.svg.png)


**Diagonalisation d'une matrice carré** 
Une matrice carrée peut être vue comme un endomorphisme allant d'un espace vectoriel $E$ vers lui même. On dira que cet endomorphisme est diagonalisable s'il existe une matrice $\boldsymbol{V}$ inversible et une matrice diagonale $\boldsymbol{\Lambda}$ telles que: 

$$\boldsymbol{A} = \boldsymbol{V} \boldsymbol{\Lambda} \boldsymbol{V}^{-1}$$



où $\boldsymbol{V}$ est une matrice dont les colonnes forment une base de $E$ et dont les éléments sont les vecteurs propres et $\boldsymbol{\Lambda}$ est une matrice dont les éléments diagonaux correspondent aux valeurs propres associées. 

**<span style='color:blue'> Matrices semblables</span>** Deux matrices $A$ et $B$ sont dites semblables s'il existe une matrice inversible $V$ telle que&nbsp;:

$$A=VBV^{-1}.$$



 ----

De plus, on peut montrer que si la matrice $\boldsymbol{A}$ est symmétrique ($\boldsymbol{A}$ = $\boldsymbol{A}^t$) à coefficients dans $\mathbb{R}$ (ces deux propriétés sont vérifiées par notre matrice de variance-covariance $\boldsymbol{\Sigma}$), la matrice de passage $\boldsymbol{V}$ est nécéssairement une matrice orthogonale ($\boldsymbol{V}^{-1} = \boldsymbol{V}^t$) et $\boldsymbol{A}$ prend alors la forme particulière suivante:

$$\begin{aligned}
\boldsymbol{A} = \boldsymbol{V} \boldsymbol{\Lambda} \boldsymbol{V}^t = \begin{bmatrix}
    \vert &   & \vert &   &  \vert \\
    \vert &   & \vert &   &  \vert \\
    \vert &   & \vert &  &  \vert \\
    \boldsymbol{v_1}   & \dots & \boldsymbol{v_i} & \dots & \boldsymbol{v_d}   \\
    \vert &   & \vert &   &  \vert \\
    \vert &  & \vert &  & \vert 
\end{bmatrix}
\begin{bmatrix}
    \lambda_1 & 0 & \dots & \dots  & 0\\
     \vdots & \ddots & \vdots & \vdots & \vdots\\
    0 & \dots & \lambda_i & \dots & 0\\
     \vdots & \vdots & \vdots & \ddots & \vdots\\
    0 & \dots & \dots & 0 &\lambda_d\\
\end{bmatrix}
\begin{bmatrix}
    \text{---} & \boldsymbol{v_1} & \text{---} \\
      & \vdots &  \\
    \text{---} & \boldsymbol{v_i} & \text{---} \\
     & \vdots &  \\
    \text{---} & \boldsymbol{v_d} & \text{---}
\end{bmatrix}
\end{aligned}$$

où $\boldsymbol{V}$ correspond cette fois à une matrice orthogonale i.e.&nbsp;:

$$\begin{aligned}
\boldsymbol{V}^t\boldsymbol{V} &= \begin{bmatrix}
    \vert &    &  \vert \\
    \boldsymbol{v_1}   &  \dots & \boldsymbol{v_d}   \\
    \vert &   & \vert 
\end{bmatrix}
\begin{bmatrix}
    \text{---} & \boldsymbol{v_1} & \text{---} \\
     & \vdots &  \\
    \text{---} & \boldsymbol{v_d} & \text{---}
\end{bmatrix}\\
&= \begin{bmatrix}
    \langle \boldsymbol{v_1}, \boldsymbol{v_1} \rangle & \dots & \langle \boldsymbol{v_1}, \boldsymbol{v_d} \rangle \\
    \vdots & \ddots & \vdots \\
      \langle \boldsymbol{v_d}, \boldsymbol{v_1} \rangle & \dots & \langle \boldsymbol{v_d}, \boldsymbol{v_d} \rangle
\end{bmatrix}\\
&= \begin{bmatrix}
    ||\boldsymbol{v_1}||_2^2=1 & \dots & 0 \\
    0 & \ddots & 0 \\
      0 & \dots & ||\boldsymbol{v_d}||_2^2=1
\end{bmatrix}
\end{aligned}$$

les vecteurs propres sont donc tous orthogonaux 2 à 2 et de norme $1$ et forment donc une base orthonormale. 


C'est ce que nous souhaitons pour notre problème. Nous voulons trouver une rotation de sorte à ce que dans la nouvelle base, qu’on appellera composantes, la variance de nos données soit le plus parfaitement décrites par les dites composantes. Nous pouvons donc reformuler de manière plus générale notre problème pour n'importe quel dimension d'entrée $d$ comme le problème d’optimisation sous contrainte suivant&nbsp;:

$$\underset{\boldsymbol{V}}{argmax}\Bigg[\frac{1}{N} \sum_{k=1}^d\sum_{i=1}^N  (z_i^k - \bar{z}^k)^2 \Bigg] = \underset{\boldsymbol{V}}{argmax} \Big[ \boldsymbol{V}^t\bar{\boldsymbol{X}}^T\bar{\boldsymbol{X}}\boldsymbol{V} \Big],\text{ s.t. }\boldsymbol{V}^t\boldsymbol{V} = \boldsymbol{I_d}$$

Et nous montrerons que la solution de ce problème consiste à trouver la base des vecteurs propres de $\boldsymbol{\Sigma} = \bar{\boldsymbol{X}}^T\bar{\boldsymbol{X}}$.

---

### B. Composition d'endormorphismes autoadjoints (symétriques)

La diagonalisation d'un endomorphisme permet de simplifier certains calculs. Nous restons ici dans le cas symétrique mais une partie du propos se généralise bien sûr au cas diagonalisable quelconque. Nous avons ainsi l'égalité suivante&nbsp;:


$$\boldsymbol{A}^2 = \boldsymbol{V}\boldsymbol{\Lambda}\underbrace{\boldsymbol{V}^t\boldsymbol{V}}_{\boldsymbol{I}}\boldsymbol{\Lambda}\boldsymbol{V}^t = \boldsymbol{V}\boldsymbol{\Lambda}^2\boldsymbol{V}^t$$


et de manière générale&nbsp;:

$$\begin{aligned}
\boldsymbol{A}^n = \boldsymbol{V} \boldsymbol{\Lambda}^n \boldsymbol{V}^t = \begin{bmatrix}
    \vert &   & \vert &   &  \vert \\
    \vert &   & \vert &   &  \vert \\
    \vert &   & \vert &  &  \vert \\
    \boldsymbol{v_1}   & \dots & \boldsymbol{v_i} & \dots & \boldsymbol{v_d}   \\
    \vert &   & \vert &   &  \vert \\
    \vert &  & \vert &  & \vert 
\end{bmatrix}
\begin{bmatrix}
    (\lambda_1)^n & 0 & \dots & \dots  & 0\\
     \vdots & \ddots & \vdots & \vdots & \vdots\\
    0 & \dots & (\lambda_i)^n & \dots & 0\\
     \vdots & \vdots & \vdots & \ddots & \vdots\\
    0 & \dots & \dots & 0 &(\lambda_d)^n\\
\end{bmatrix}
\begin{bmatrix}
    \text{---} & \boldsymbol{v_1} & \text{---} \\
      & \vdots &  \\
    \text{---} & \boldsymbol{v_i} & \text{---} \\
     & \vdots &  \\
    \text{---} & \boldsymbol{v_d} & \text{---}
\end{bmatrix}
\end{aligned}$$

On remarque ainsi que si $A$ est symétrique, alors $A^n$ possède la même matrice de changement de base et ses valeurs propres sont les mêmes à la puissancec $n$.

Notez que n'importe quel vecteur $\boldsymbol{x} \in \mathbb{R}^d$ peut se décomposer dans la base des vecteurs propres&nbsp;:

$$\boldsymbol{x} = \sum_{k=1}^d \underbrace{\langle \boldsymbol{v_k}, \boldsymbol{x} \rangle}_{z_k} \boldsymbol{v_k}$$

---

**<span style='color:blue'> Question </span>** 
**Montrez que $\boldsymbol{A}\boldsymbol{x} =  \sum_{k=1}^d \lambda_k  z_k \boldsymbol{v_k}$.**


 ----


**<span style='color:blue'> Question</span>** 
**En utilisant la propriété de la mise à la puissance, donnez l'expression de $\boldsymbol{A}^n\boldsymbol{x}$.**


 ----


## III. Lien entre directions de variance maximale et vecteurs propres

**<span style='color:blue'> Proposition</span>** 
La direction de variance maximale dans $\boldsymbol{X}$ correspond au vecteur propre associé à la plus grande valeur propre de $\boldsymbol{\Sigma}=\boldsymbol{X}^T\boldsymbol{X}$.



 ----

**<span style='color:orange'> Preuve</span>** 
$$\begin{aligned}\boldsymbol{v}^t\bar{\boldsymbol{X}}^T\bar{\boldsymbol{X}}\boldsymbol{v} &= \boldsymbol{v}^t\boldsymbol{\Sigma}\boldsymbol{v} = \Big\langle\boldsymbol{v}, \boldsymbol{\Sigma}\boldsymbol{v} \Big\rangle\\ & = \Bigg\langle\boldsymbol{v}, \Bigg[\sum_{k=1}^d \lambda_k    \langle \boldsymbol{v}, \boldsymbol{v_k}\rangle   \boldsymbol{v_k}\Bigg] \Bigg\rangle \\ &= \sum_{k=1}^d \lambda_k    \langle \boldsymbol{v}, \boldsymbol{v_k}\rangle \langle \boldsymbol{v}, \boldsymbol{v_k}\rangle  \\ &\leq \lambda_{\max} \underbrace{\sum_{k=1}^d     \langle \boldsymbol{v}, \boldsymbol{v_k}\rangle^2}_{=\lVert\boldsymbol{v}\rVert_2^2 = 1}\\ &   = \lambda_{\max}\end{aligned}$$

L'égalité est ainsi obtenue pour le vecteur propre associé à $\lambda_{max}$. 



 ----
Nous avons donc&nbsp;:


$$\lambda_{\max}=\boldsymbol{v}_{\max}
^t\bar{\boldsymbol{X}}^T\bar{\boldsymbol{X}}\boldsymbol{v}_{\max}$$

Nous devons partir à la recherche de ce vecteur propre particulier pour résoudre notre problème. Pour cela nous allons voir une méthode itérative pratique permettant de retrouver le vecteur associé à la plus grande valeur propre d'une matrice&nbsp: **l'algorithme des puissances itérées**.

## IV. Calculer les vecteurs et valeurs propres

### A. L'algorithme des puissances itérées

En réutilisant les propriétés vues à propos des matrices diagonalisables, nous pouvons essayer de construire un algorithme qui nous permettra de calculer le vecteur propre associé à la plus grande valeur propre&nbsp;:

$$\underbrace{\boldsymbol{A}\boldsymbol{A}\dots\boldsymbol{A}}_{\times n}\boldsymbol{v} = \boldsymbol{A}^n\boldsymbol{v} =  \sum_{k=1}^d (\lambda_k)^n  \langle \boldsymbol{v_k}, \boldsymbol{v}\rangle   \boldsymbol{v_k}$$

Et on voit que c'est la contribution du vecteur propre associée à la plus forte valeur propre qui va dominer assymptotiquement. Plus formellement, si on écrit les valeurs propres sous la formes $\lambda_k = \lambda_{\max}\frac{\lambda_{k}}{\lambda_{\max}}$&nbsp;:

$$\boldsymbol{A}^n\boldsymbol{v} = (\lambda_{\max})^n \sum_{k=1}^d \Big(\frac{\lambda_{k}}{\lambda_{\max}}\Big)^n  \langle \boldsymbol{v_k}, \boldsymbol{v}\rangle   \boldsymbol{v_k},$$

on obtient $\lim_{n \rightarrow \infty} (\frac{\lambda_{k}}{\lambda_{\max}})^n = 1$ si $\lambda_{k} = \lambda_{\max}$ et $0$ sinon. Ainsi&nbsp;:


$$\lim_{n \rightarrow \infty}\frac{\boldsymbol{A}^n\boldsymbol{v}}{\lVert\boldsymbol{A}^n\boldsymbol{v}\rVert_2} = \boldsymbol{v_{\max}}$$


En multipliant itérativement (presque) n'importe quel vecteur d'entrée par notre application linéaire, nous construisons une suite dont l'expression normalisée converge asymptotiquement vers le vecteur propre associé à la valeur propre la plus forte. C'est exactement ce qu'on fait en pratique en alternant multiplication et normalisation afin d'éviter un $\texttt{overflow}$ de nos variable. L'initialisation est faite aléatoirement&nbsp;:

\begin{equation*}
\boldsymbol{v}(0) \leftarrow \mathcal{N}\Big(\boldsymbol{0}, \boldsymbol{I}\Big).
\end{equation*}

Et chaque itération a la forme suivante&nbsp;:

\begin{equation*}
\boldsymbol{v}(n+1) =  \frac{\boldsymbol{A}\boldsymbol{v}(n)}{\lVert\boldsymbol{A}\boldsymbol{v}(n)\rVert_2} 
\end{equation*}


**<span style='color:blue'> Question </span>** 
**Maintenant que nous avons un algorithme pour trouver la composante principale, comment faire pour trouver les autres ?**


 ----


### B. En pratique

**<span style='color:blue'> Exercice</span>** 
**Completer le code des méthodes $\text{power_iteration}$ et $\text{fit}$ de la classe *ACP* ci-dessous qui implemente le calcul d'une ACP. La méthode $\text{power_iteration}$ doit retourner un tuple contenant le vecteur et sa valeur propre associée. La méthode $\text{fit}$ doit exécuter les puissances itérées autant de fois que demandé. C'est-à-dire $\texttt{n}\_\texttt{components}$ fois.**


 ----

In [ ]:
class ACP(object):
    def __init__(self, n_components=2, n_iter=1):
        self.n_components = n_components
        self.n_iter = n_iter

    def order_vectors(self,):
        assert self.eigen_vectors is not None or self.eigen_values is not None, "Must be fitted before"
        idx = np.argsort(-self.eigen_values)
        self.eigen_values = self.eigen_values[idx]
        self.eigen_vectors = self.eigen_vectors[:, idx]
    
    #### Complete the code here #### or die #######################################
    def power_iteration(self, A):
        ...
        ...
        return vector, value

    def fit(self, X, ):
        self.X = X
        self.N = X.shape[0]
        self.d = X.shape[1]

        self.mean = X.mean(axis=0)
        X_n = self.X - self.mean
        self.sigma = np.dot(X_n.T, X_n)/float(self.N)

        self.eigen_vectors = np.zeros((self.d, self.n_components)) 
        self.eigen_values = np.zeros(self.n_components) 
        ...
        ...
        ...
        ...
        self.order_vectors()
    ###############################################################################
    
            
    def zeros_pad(self, Z, k):
        return np.c_[Z, np.zeros((Z.shape[0], self.d - k))]
        
    def transform(self, X, k = None):
        #### Complete the code here #### or die #######################################
        # pour un exercice plus bas
        #
        #
        ###############################################################################
        pass
        
    
    def inverse_transform(self, Z, k = None):
        #### Complete the code here #### or die #######################################
        # pour un exercice plus bas
        #
        #
        ###############################################################################
        pass

    def compress(self, X, k = None):
        return self.inverse_transform(self.transform(X, k), k)


Appliquons notre méthode à nos données. On affichera le repère associé aux composantes principales trouvés par notre méthode&nbsp;:

In [ ]:
acp = ACP(n_components=2, n_iter=10)
acp.fit(X)

plot(X, vec = acp.eigen_vectors.T)
plt.show()


En pratique, les calculs ne sont pas aussi directs et certaines décompositions sont utilisées afin d'accélérer et de stabiliser les calculs.

## V. Calcul des vecteurs de représentation (codage) et reconstructrion (décodage) 

### A. Compression

Nous venons donc de trouver une base telle que les projections des données dans ce nouveau système de coordonnées ont une variance maximale. Rappelons nous que l'objectif de l'apprentissage non-supervisé de représentation à pour objectif de trouver un mapping, une fonction de codage $\Phi : \boldsymbol{x} \rightarrow \boldsymbol{z} = \Phi(\boldsymbol{x}) :  \mathbb{R}^d \rightarrow \mathbb{R}^K$, et dans notre cas nous pouvons définir $\Phi$ comme l'application linéaire suivante&nbsp;:


$$\boldsymbol{z} = \hat{\boldsymbol{V}}_K^T \boldsymbol{x} \in \mathbb{R}^K$$

où $\hat{\boldsymbol{V}}_K^T \in \mathbb{R}^{d \times K}$ correspond à la matrice  $\boldsymbol{V}$ de laquelle on a retirée les $d-K$ derniers vecteurs colonnes (on part du principe que les vecteur colonne sont triés par ordre décroissant de leur valeur propre associée). Nous avons donc&nbsp;:


$$\hat{\boldsymbol{V}}_K = 
\begin{bmatrix}
    \vert &   & \vert    \\
    \vert &   & \vert    \\
    \vert &   & \vert \\
    \boldsymbol{v_1}   & \dots & \boldsymbol{v_K}   \\
    \vert &   & \vert    \\
    \vert &  & \vert 
\end{bmatrix}$$


**<span style='color:blue'> Question</span>** 
**Exprimez une application "inverse" (fonction de décodage), qu'on notera $\Psi$, qui permettrait de reprojeter la representation $\boldsymbol{z}$ dans l'espace d'origine. On notera $\hat{\boldsymbol{x}} = \Psi(\boldsymbol{z})$ la reconstruction du vecteur d'origine.**


 ----

Il est ainsi évident que si toutes les composantes sont conservées, on retrouve $\hat{\boldsymbol{x}} = (\Psi \circ \Phi) ( \boldsymbol{x}) =  \underbrace{\boldsymbol{V}\boldsymbol{V}^t}_{\boldsymbol{I_d}} \boldsymbol{x} = \boldsymbol{x}$.

On notera que comme $\hat{\boldsymbol{V}}_K$ avec $K \leq d$ n'est pas carrée de rang plein, elle n'est pas inversible, et l'application $\Phi$ n'est pas bijective. Ainsi, $\Psi$ ne peut pas être l'inverse de $\Phi$. Nous avions donc plusieurs choix pour $\Psi$. Mais nous pouvons montrer que celui que nous avons fait minimise l'erreur de décodage&nbsp;: c'est une propriété souhaitable pour ce genre de problème en basse dimension.

---
**<span style='color:blue'> Exercice</span>** 
**Dans le code de l'ACP ci-dessus, complétez les methodes $\text{transform}$ et $\text{inverse_transform}$.**


 -------

Appliquons ensuite le code ci dessous pour visualiser les entrées reconstruites (en rouge) par dessus les données d'entrée (en bleu).

In [ ]:
X_rec = acp.compress(X, k=1)  # on ne projette que sur une composante

plot(X, vec = acp.eigen_vectors.T, X_rec = X_rec)
plt.show()


### B. Interprétation de l'erreur de reconstruction moyenne

Nous allons voir ici que la fonction de codage/décodage correspondant à l'ACP correspond à un endomorphisme qui est un projecteur orthogonal. De plus la différence moyenne entre le vecteur d'origine et son projeté orthogonal possède la propriété intéressante suivante&nbsp;:

$$\begin{aligned}
        \text{err} &= \frac{1}{N}\sum_i^N||\boldsymbol{x}_i - \hat{\boldsymbol{V}}_K\hat{\boldsymbol{V}}_K^t \boldsymbol{x}_i||_2^2\\&=
        \frac{1}{N}\sum_i^N||\sum_{k=1}^d\langle\boldsymbol{x}_i, \boldsymbol{v}_k \rangle \boldsymbol{v}_k - \sum_{k=1}^K\langle\boldsymbol{x}_i, \boldsymbol{v}_k \rangle \boldsymbol{v}_k||_2^2\\ &=
         \frac{1}{N}\sum_i^N||\sum_{k=K+1}^d\langle\boldsymbol{x}_i, \boldsymbol{v}_k \rangle \boldsymbol{v}_k||_2^2 =
         \frac{1}{N}\sum_i^N \sum_{k=K+1}^d\langle\boldsymbol{x}_i, \boldsymbol{v}_k \rangle^2 \\ &=
         \frac{1}{N}\sum_{k=K+1}^d \boldsymbol{v}_k^t \boldsymbol{X}^T\boldsymbol{X}\boldsymbol{v}_k = \sum_{k=K+1}^d \boldsymbol{v}_k^t \boldsymbol{\Sigma}\boldsymbol{v}_k\\ =&
         \sum_{k=K+1}^d \lambda_k \xrightarrow{N\rightarrow\infty}\sum_{k=K+1}^d \text{Var}_{\boldsymbol{z}}[z_k]
\end{aligned}$$

Nous pouvons observer qu'à l'étape 3, le vecteur différence est forcément un vecteur de $\mathbb{R}^d$ et est orthogonale à tous les vecteurs propres sélectionnés dans $\hat{\boldsymbol{V}}_K$. $\hat{\boldsymbol{V}}_K\hat{\boldsymbol{V}}_K^T$ est donc un projecteur orthogonal sur le sous espace propre correspondant aux composantes sélectionnées. Il est celui qui minimise l'erreur de reconstruction. De plus nous constatons que cette erreur a une norme au carré moyenne qui est une estimation de la somme des variances dans les directions non pertinentes. 


En pratique on pourra donc choisir le nombre de composante à conserver $K$ de sorte à ce que la qualité de la reconstruction souhaitée soit supérieure à un certain seuil $\tau$&nbsp;:
\begin{equation*}
   \frac{\sum_{k=1}^K \lambda_k}{\sum_{k=1}^d \lambda_k} \geq \tau \in [0,1]
\end{equation*}

**<span style='color:blue'> Exercice</span>** 
**Completez les fonction $\text{compute_cumulative_var}$ et $\text{find_thresholded_cumulative}$ qui calcul respectivement l quantité exprimez précédement, et détermine le nombre de composante minimale pour avoir un *RMSE* d'au moins $\tau$ (*tresh*).**


 ----

In [ ]:
def compute_error(X, X_rec):
    return np.linalg.norm(X - X_rec)/np.linalg.norm(X)

def compute_cumulative_var(eigen_values, k):
    #### Complete the code here #### or die #######################################
    ...
    ...
    ###############################################################################

def find_thresholded_cumulative(eigen_values, thresh = 0.9):
    #### Complete the code here #### or die #######################################
    ...
    ...
    ###############################################################################


On peut maintenant construire un jeu de donnée de plus haute dimension et tester notre code:

In [ ]:
X = sample_multivariate_gaussian_data(500, 128)
print(X.shape)

acp = ACP()
acp.fit(X)

cumul = [compute_cumulative_var(acp.eigen_values, k=k)for k in range(X.shape[1])]

K = find_thresholded_cumulative(acp.eigen_values, thresh = 0.95)
print(K, np.float64(K)/X.shape[1])

plt.figure()
plt.plot([i for i in range(1, len(cumul)+1)], cumul, 
         label='Erreur de reconstruction')
plt.legend()
plt.show()


On voit ici que sulement 0.7% des composantes contiennent 95% de l'information.

## V. Compression d'une base d'images

### A. Les données et notre ACP

Dans cette section nous allons tester notre algorithme d'ACP sur des données d'images brutes en considérant chaque image de dimension $(w \times h)$ comme un vecteur $\boldsymbol{x} \in \mathbb{R}^{wh}$. Nous procéderons sur une base d'images de visages centrés en niveau de gris&nbsp;: la base $\text{Oliveti Faces}$. Commençons dans un premier temps par charger les données, les afficher et appliquons notre algorthme d'ACP (on prendra plutôt la version $\text{np.linalg.eig}$ pour calculer les vecteur propres qui est beaucoup plus stables et efficace computationellement que la notre).  

In [ ]:
class ACP(object):
    def __init__(self):
        pass

    def order_vectors(self,):
        assert self.eigen_vectors is not None or self.eigen_values is not None, "Must be fitted before"
        idx = np.argsort(-self.eigen_values)
        self.eigen_values = self.eigen_values[idx]
        self.eigen_vectors = self.eigen_vectors[:, idx]

    def fit(self, X, ):
        self.X = X
        self.N = X.shape[0]
        self.d = X.shape[1]

        self.mean = X.mean(axis=0)
        X_n = self.X - self.mean
        self.sigma = np.dot(X_n.T, X_n)/float(self.N)

        self.eigen_values, self.eigen_vectors = np.linalg.eig(self.sigma)
        self.eigen_values = self.eigen_values.real
        self.eigen_vectors = self.eigen_vectors.real
        self.order_vectors()
        
    def zeros_pad(self, Z, k):
        return np.c_[Z, np.zeros((Z.shape[0], self.d - k))]
        
    def transform(self, X, k = None):
        assert self.eigen_vectors is not None, "You need to fit the model first !"
        k = self.n_components if k is None else k
        return np.dot(X - self.mean, self.eigen_vectors[:, :k])
    
    def inverse_transform(self, Z, k = None):
        assert self.eigen_vectors is not None, "You need to fit the model first !"
        k = self.n_components if k is None else k
        return np.dot(self.zeros_pad(Z, k), self.eigen_vectors.T) + self.mean

    def compress(self, X, k = None):
        return self.inverse_transform(self.transform(X, k), k)

In [ ]:
from sklearn import datasets
 
# Load data
lfw_dataset = datasets.fetch_olivetti_faces() #datasets.fetch_lfw_people(min_faces_per_person=500)
 
_, h, w = lfw_dataset.images.shape
X = lfw_dataset.data
print(X.shape)

In [ ]:
# Visualization
def plot_images(images, h, w, rows=8, cols=8, title=None):
    plt.figure(figsize=(64,64))
    for i in range(rows * cols):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        if title is not None:
            plt.title(title + " : " + str(i+1))
        plt.xticks(())
        plt.yticks(())

def plot_images_one_by_one(images, original, h, w, step=1, title=None):
    for i in range(images.shape[0]):
        plt.figure(figsize=(6, 12))

        plt.subplot(1, 2, 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.xticks(())
        plt.yticks(())
        if title is not None:
            plt.title(title + " : " + str(i*step+1))

        plt.subplot(1, 2, 2)
        plt.imshow(original.reshape((h, w)), cmap=plt.cm.gray)
        plt.xticks(())
        plt.yticks(())
        if title is not None:
            plt.title('Original image')
        

            
plot_images(X, h, w)

In [ ]:
acp = ACP()
acp.fit(X)

### B. Affichage des variances cumullées et calcul du seuil de reconstruction

Nous calculons ensuite le nombre de composantes à conserver pour obtenir une qualité de reconstruiuction moyenne de $95\%$ de $\text{RMSE}$. On voit ici que l'on peut ne conserver que $2 \%$ des variables d'entrées pour avoir une qualité moyenne de reconstruction de $95 \%$.

In [ ]:
cumul = [compute_cumulative_var(acp.eigen_values, k=k)for k in range(X.shape[1])]

K = find_thresholded_cumulative(acp.eigen_values, thresh = 0.95)
print(K, float(K)/X.shape[1])

plt.figure()
plt.plot([i for i in range(1, len(cumul)+1)], cumul, 
         label='Erreur de reconstruction')
plt.legend()
plt.show()


### C. Affichage des vecteur propres (les eigen faces)

Affichons les vecteurs propres appris sur ce jeu de données. On remarquera qu'on peut interpreter ces vecteurs propres coommes des images et donc qu'on peut les afficher comme tel. On affichera un sous ensemble dans un soucis de lisibilité. Une manière d'interpreter ces images est donc que tout visage de la base peut être reconstruit sans erreur comme une somme pondérée des ces visages "élémentaires".

In [ ]:
plot_images(acp.eigen_vectors.T, h,w)


### D. Visualisation perceptuelle de la qualité de reconstruction

Nous pouvons aussi nous amuser visualiser différentes versions reconstruites d'une même image en ne conservant qu'une certaine proportion des composantes $k \in [1, K]$. On voit ici que, perceptuellement, même sans aller jusqu'au nombre de composantes seuil définit précédement, on peut très rapidmeent converger vers l'image originale avec peu de composantes.

In [ ]:
idx = 14#np.random.randint(X.shape[0])
image = X[idx].reshape(1,X.shape[1])
compressed_images = np.array([acp.compress(image, k=k) for k in range(0, K, 20)])

plot_images_one_by_one(compressed_images, image, h, w, step=20, 
                       title="Progressive reconstruction \n n_components")


### E. Essayons les mêmes étapes sur mnist

In [ ]:
from sklearn.datasets import fetch_openml
X, _ = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)

plot_images(X, 28, 28)

In [ ]:
acp = ACP()
acp.fit(X)

#errors = [compute_error(X, x_rec) for x_rec in [acp.compress(X, k=k) for k in range(X.shape[1])]]
cumul = [compute_cumulative_var(acp.eigen_values, k=k)for k in range(X.shape[1])]
plt.figure()
plt.plot(
    [i for i in range(1, len(cumul)+1)], cumul, 
    label='Erreur de reconstruction'
)
plt.legend()
plt.show()


In [ ]:
plot_images(acp.eigen_vectors.T, 28, 28)


In [ ]:
K = find_thresholded_cumulative(acp.eigen_values, thresh = 0.90)
print(K, float(K)/X.shape[1])

idx = 142
image = X[idx].reshape(1,X.shape[1])

compressed_images = np.array([acp.compress(image, k=k) for k in range(0, K, 10)])

plot_images_one_by_one(
    compressed_images, image, 28, 28, step=10, 
    title="Progressive reconstruction \n n_components"
)
